# Face Recognition Test

## 1. Importar as Bibliotecas

### 1.1. Instalação
Caso não as possua basta intalar pelo pip:

In [44]:
#%pip install opencv-python imgbeddings psycopg2-binary

### 1.2. Importando

In [45]:
import cv2  #biblioteca opencv-python
from imgbeddings import imgbeddings
import psycopg2 #biblioteca psycopg2-binary

### 1.3. Importando bibliotecas padrões

In [46]:
import numpy as np
import os
from PIL import Image

## 2. Algoritmo de Reconhecimento 

### 2.1. Aquisição do Haarcascade
Basta baixá-la pelo link: https://github.com/kipr/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml <br>
Depois de baixar, coloque no mesmo repositório desse código. <br>
Também será necessário criar uma pasta chamada stored-faces para armazenar cada imagem de face.

In [66]:
haar_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml") #carregando o algoritmo no OpenCV
file_name = "test-image.jpeg"    #carregar a imagem para fazer o teste do algoritmo
img = cv2.imread(file_name, 0)  #lendo imagem com OpenCV
gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) #transformar a imagem monocromática(preto e branco)

faces = haar_cascade.detectMultiScale(  #detectando faces
    gray_img, scaleFactor=1.05, minNeighbors=2, minSize=(100, 100)  #argumentos podem ser alterados aqui, preciso estudar melhor eles
)

i = 0   #iterador para cade face
for x, y, w, h in faces:
    cropped_image = img[y : y + h, x : x + w]   #cortar a imagem para cada face
    target_file_name = 'stored-faces/' + str(i) + '.jpg'    #salvar foto de face, uma a uma
    cv2.imwrite(
        target_file_name,
        cropped_image,
    )
    i = i + 1

eae


## 3. Criando Embeddings

### 3.1. Criando uma conta aiven
Criaremos uma conta para conectar no aiven para conectar ao postgree, se preferir pode fazer no seu pc. <br>
Site: https://console.aiven.io/ <br>
Crie um projeto com o postgree, depois vá em quick connect e copie o comando psql para rodar no seu terminal(deve esperar até o servidor concluir o building). <br>
Caso não possua, instale o postgree client:

In [48]:
#%sudo apt-get install postgresql-client

###  3.2. Criando Vetor e Tabela no Servidor

Ao acessar seu postgree crie as seguintes estruturas: <br>
CREATE EXTENSION vector; <br>
CREATE TABLE pictures(picture text PRIMARY KEY, embedding vector(768));

Criadas as tabelas no servidor, vamos conectar nosso programa ao servidor, para isso copie o "SERVICE URI" no aiven

### 3.3. Enviando embeddings ao Servidor

In [50]:
#substitua pela sua conexão aqui
conn = psycopg2.connect("")    #conectando ao servidor

for filename in os.listdir("stored-faces"):
    img = Image.open("stored-faces/" + filename)    #carregar cada face na pasta
    ibed = imgbeddings()    #carregar o imgbeddings
    embedding = ibed.to_embeddings(img) #calcular os embeddings de cada foto
    cur = conn.cursor()
    #cur.execute("INSERT INTO pictures values (%s,%s)", (filename, embedding[0].tolist()))   #salvar os calculos dos embeddings no servidor postgree
    print(filename)
conn.commit()

0.jpg
2.jpg
3.jpg
1.jpg


## 4. Calculando Embedding para Foto Nova

In [51]:
file_name = "solo-image.jpeg"  #imagem nova para testar se é a mesma pessoa
img = Image.open(file_name)
ibed = imgbeddings()
embedding = ibed.to_embeddings(img)

## 5. Procurando Similaridades entre o Postgresql e a Nova Foto

In [ ]:
from IPython.display import Image, display

cur = conn.cursor()
string_representation = "["+ ",".join(str(x) for x in embedding[0].tolist()) +"]"
cur.execute("SELECT * FROM pictures ORDER BY embedding <-> %s LIMIT 1;", (string_representation,))
rows = cur.fetchall()
for row in rows:
    display(Image(filename="stored-faces/"+row[0]))
cur.close()